In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import supervision as sv
from ultralytics import solutions




# Constante para tamanho da janela deslizante
WINDOW_SIZE = 512

# Classe que realiza o processamento de cada janela
class ImageProcessor:
    def __init__(self, image_path, window_size=WINDOW_SIZE):
        self.image_path = image_path
        self.window_size = window_size
        self.image = cv2.imread(self.image_path)
        self.processed_image = None
        self.processed_windows = []  # Lista para armazenar os quadrados processados
        self.object_counts = []  # Lista para armazenar a contagem de objetos em cada quadrado

    def normalize_square(self, square):
      """
      Normaliza um quadrado individualmente.
      Combina padronização de cor, remoção de ruído e ajuste de contraste.
      """
    
      # Padroniza a cor do quadrado
      
      
      square = self.standardize_color(square)
      return square

  

    def standardize_color(self, square):
      """
      Padroniza as cores de um quadrado usando a abordagem General Gray World.
      """
      # Converte para o espaço RGB para calcular a iluminação média
      mean_rgb = np.mean(square, axis=(0, 1))  # Média por canal RGB
      
      # Calcula o fator de escala com base na média do canal
      scale_factors = mean_rgb.mean() / mean_rgb
      scale_factors = np.clip(scale_factors, 2.4, 3.8)  # Limitar valores extremos
      
      # Normaliza os canais RGB
      normalized_square = np.zeros_like(square, dtype=np.float32)
      for c in range(3):  # Para cada canal (R, G, B)
          normalized_square[:, :, c] = square[:, :, c] * scale_factors[c]
      
      # Garante que os valores estejam no intervalo [0, 255]
      normalized_square = np.clip(normalized_square, 0, 255).astype(np.uint8)

      return normalized_square

    def normalize_grayscale(self, square):
      # Convert image to grayscale
      gray_image = cv2.cvtColor(square, cv2.COLOR_BGR2GRAY)

      # Normalize grayscale image
      normalized_gray_image = cv2.normalize(
          gray_image, None, alpha=0, beta=80, norm_type=cv2.NORM_MINMAX)

      # Convert normalized grayscale image back to color
      normalized_color_image = cv2.cvtColor(
          normalized_gray_image, cv2.COLOR_GRAY2BGR)

      return normalized_color_image
    
    


    def apply_window_pipeline(self):
      """Etapa 2: Processamento com janela deslizante sem interseções e preenchimento."""
      # Dimensões da imagem
      img_height, img_width, _ = self.image.shape

      # Calcula os novos tamanhos com preenchimento
      padded_height = ((img_height + self.window_size - 1) // self.window_size) * self.window_size
      padded_width = ((img_width + self.window_size - 1) // self.window_size) * self.window_size

      # Preenche a imagem com bordas negras
      padded_image = np.zeros((padded_height, padded_width, 3), dtype=self.image.dtype)
      padded_image[:img_height, :img_width] = self.image

      # Prepara imagem processada
      processed_image = np.zeros_like(padded_image)

      # Processa cada janela
      for y in range(0, padded_height, self.window_size):
          for x in range(0, padded_width, self.window_size):
              window = padded_image[y:y + self.window_size, x:x + self.window_size]
              processed_window = self.process_window(window)
              self.processed_windows.append(processed_window)
              processed_image[y:y + self.window_size, x:x + self.window_size] = processed_window
              object_count, image_counted = self.count_objects(processed_window)
              processed_image[y:y + self.window_size, x:x + self.window_size] = image_counted
              self.object_counts.append(object_count)

      # Remove o preenchimento da imagem processada
      self.processed_image = processed_image[:img_height, :img_width]

    def process_window(self, window):
        """Executa as etapas do pipeline em uma janela individual."""
        normalized_window = self.normalize_square(window)
        return normalized_window

    def count_objects(self, square):
        # """Conta o número de objetos na imagem."""

        # # Contar objetos na image
        # model_path = '/media/williancaddd/CODES/fiotec/eggs-count-algorithms/Labels Eggs.v9i.yolov11/runs/detect/train/weights/best.pt'
        # region=[(0, 0), (512, 0), (512, 512), (0, 512)]
        # counter = solutions.ObjectCounter(model=model_path, region=region, show=False)
        # image_contend = counter.count(square)
        # num_eggs = len(counter.boxes)
    

        return 0, square    

    def show_results(self):
        """Exibe a imagem original, a imagem final processada e os quadrados agrupados."""
        fig, axes = plt.subplots(3, 1, figsize=(20, 30))

        # Imagem original
        axes[0].imshow(cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Imagem Original")
        axes[0].axis("off")

        # Imagem processada final
        axes[1].imshow(cv2.cvtColor(self.processed_image, cv2.COLOR_BGR2RGB))
        axes[1].set_title("Imagem Processada Final")
        axes[1].axis("off")

        # Quadrados processados agrupados
        num_windows = len(self.processed_windows)
        num_cols = 4
        num_rows = (num_windows + num_cols - 1) // num_cols
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 2.5))

        for idx, processed_window in enumerate(self.processed_windows):
            row = idx // num_cols
            col = idx % num_cols
            ax = axes[row, col]
            ax.imshow(cv2.cvtColor(processed_window, cv2.COLOR_BGR2RGB))
            ax.set_title(f"Quadrado {idx + 1}\nObjetos: {self.object_counts[idx]}")
            ax.axis("off")

        for idx in range(len(self.processed_windows), num_rows * num_cols):
            row = idx // num_cols
            col = idx % num_cols
            axes[row, col].axis("off")

        plt.tight_layout()
        plt.show()

    def show_specific_windows(self, indices):
      """
      Exibe quadrados processados com base nos índices fornecidos.
      """
      fig, axes = plt.subplots(1, len(indices), figsize=(10 * len(indices), 10))

      if len(indices) == 1:  # Caso apenas um quadrado seja exibido
          axes = [axes]

      for i, idx in enumerate(indices):
          if idx < len(self.processed_windows):  # Verifica se o índice é válido
              window = self.processed_windows[idx]
              axes[i].imshow(cv2.cvtColor(window, cv2.COLOR_BGR2RGB))
              axes[i].set_title(f"Quadrado {idx + 1}\nObjetos: {self.object_counts[idx]}")
              axes[i].axis("off")
          else:
              print(f"Índice {idx} está fora do intervalo de quadrados processados.")

      plt.tight_layout()
      plt.show()



    def process(self):
        """Executa todo o pipeline."""
        self.apply_window_pipeline()


# Caminho da imagem de entrada
image_path = '/media/williancaddd/CODES/fiotec/eggs-count-algorithms/base-4/paleta5.png'

# Execução do processamento
processor = ImageProcessor(image_path)

processor.process()
processor.show_specific_windows([15, 15])  # Subtraímos 1 do índice, pois listas começam do 0

# Exibição dos resultados
processor.show_results()

# total 
print("Total de ovos: ", sum(processor.object_counts))


